In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import math
import warnings
warnings.filterwarnings('ignore')
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
data=pd.read_csv('marketing_customer_analysis_labFinal.csv')


In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data=data.drop(['Customer'], axis=1)

In [ ]:
data = data.drop_duplicates()

In [ ]:
data.shape


In [ ]:
data.columns = [val.lower().replace(' ','_') for val in data.columns]
data.columns

In [ ]:
data.isna().sum()
#there are no NaN values
pd.isna(data['income']).sum()   

In [ ]:
X_cat=data.select_dtypes(include=np.object)


In [ ]:
X_cat = X_cat.drop(['effective_to_date'], axis=1)

In [ ]:
X_num=data.select_dtypes(include = np.number)

In [ ]:
for i in data.columns:
    print(data[i].unique())

In [ ]:
data['effective_to_date']= pd.to_datetime(data['effective_to_date'],errors='coerce')

In [ ]:
data.dtypes

In [ ]:
data.info(memory_usage='deep')

In [ ]:
for i in X_num.columns:
    sns.distplot(X_num[i])
    plt.show()

In [ ]:
data.describe()

In [ ]:
correlations_matrix= X_num.corr()
mask = np.zeros_like(correlations_matrix)
mask[np.triu_indices_from(mask)] = True
fig, ax = plt.subplots(figsize=(10, 8))
ax = sns.heatmap(correlations_matrix, mask=mask, annot=True)
plt.show()

In [ ]:
#testdata['customer_lifetime_value']=list(map(log_transfom_clean_, testdata['customer_lifetime_value']))

In [ ]:
X_num.head()

Removing outliers

In [ ]:
def remove_outliers(df, feature, factor):
    iqr = np.percentile(df[feature],75) - np.percentile(df[feature],25)
    upper_limit = np.percentile(df[feature],75) + factor*iqr
    lower_limit = np.percentile(df[feature],25) - factor*iqr
    df=df[(df[feature]>lower_limit) & 
                  (df[feature]<upper_limit)]

In [ ]:
X_num.head()

In [ ]:
mean_income = np.mean(X_num['income'])
X_num=X_num.replace({'income': {0: mean_income}}) 
X_num.head()

In [ ]:
for i in list(X_num.columns):
    remove_outliers(X_num, i, 100)

In [ ]:
from scipy import stats
def feat_log_box(df, feature, bins):
    """makes a log Transformation and a boxcox transformation of a feature of a dataframe. \n
    0 values are replaced with the mean in the transformation: \n
    plots 3 distplots next to each other for comparison \n
    tales 3 arguments (df, feature, bins)"""
    df[feature+'_log'] = list(map(lambda x: np.log(x) if np.isfinite(x) else np.NAN, df[feature]))
    df[feature+'_log'] = df[feature+'_log'].fillna(np.mean(df[feature+'_log']))
    df[feature+'_boxcox'] = np.where(df[feature]<=0,0,df[feature])
    mean = np.sum(df[feature+'_boxcox'])/len(df[df[feature+'_boxcox']>0])
    df[feature+'_boxcox'] = df[feature+'_boxcox'].replace(0,mean)
    xt, lmbda = stats.boxcox(df[feature+'_boxcox'])
    df[feature+'_boxcox'] = xt
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (15,4))
    sns.distplot(df[feature], bins, ax=ax1)
    ax1.set_title(feature)
    sns.distplot(df[feature+'_log'], bins, ax=ax2)
    ax2.set_title(feature + '_log')
    sns.distplot(xt, bins, ax=ax3)
    ax3.set_title(feature + '_boxcox')

In [ ]:
for i in list(X_num.columns):
    feat_log_box(X_num, i, 100)

In [ ]:
X_num.head()

In [ ]:
X_num = X_num.drop(['customer_lifetime_value', 'customer_lifetime_value_log', 'income', 'income_log', 'monthly_premium_auto_log', 'monthly_premium_auto', 'months_since_last_claim_log', 'months_since_last_claim_boxcox'], axis=1)

In [ ]:
Y = X_num['total_claim_amount']
X_num = X_num.drop(['total_claim_amount'], axis=1)

In [ ]:
X_num.head()

In [ ]:
from sklearn.preprocessing import Normalizer #normalizer is an alternative to Standardscaler. Norm distributes between 0 and 1, Sscaler distributes between -1 an +1
fitted = Normalizer().fit(X_num)
x_normalized = fitted.transform(X_num)
print(x_normalized.shape)
x_normalized=pd.DataFrame(x_normalized)

In [ ]:
Y=pd.DataFrame(Y)

In [ ]:
yfitted = Normalizer().fit(Y)
y_normalized = yfitted.transform(Y)
print(y_normalized.shape)
y_normalized=pd.DataFrame(y_normalized)

In [ ]:
x_normalized.head()

In [ ]:
y_normalized.head()

dealing with categoricals now

In [ ]:
X_cat.head()

In [ ]:
encoded=pd.get_dummies(X_cat, drop_first=True)
print(encoded)
type(encoded)
encoded.shape

In [ ]:
X = np.concatenate((x_normalized, encoded), axis=1)
pd.DataFrame(X).head()
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=100)

In [ ]:
from sklearn import linear_model

In [ ]:
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)
predictions  = lm.predict(X_test)
r2_score(y_test, predictions)

In [ ]:
X_train.shape

In [ ]:
pd.DataFrame(y_train).shape

In [ ]:
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)
model

In [ ]:
pd.DataFrame(X_test)

In [ ]:
len(lm.predict(X_test))

In [ ]:
mse = mean_squared_error(y_test, predictions)
print(mse)

In [ ]:
rmse = math.sqrt(mse)
print(rmse)

In [ ]:
r2 = r2_score(y_test, predictions)
r2

In [ ]:
n = len(X_test) 
p = X_test.shape[1]

In [ ]:
adj_r2 = 1-((1-r2)*(n-1)/(n-p-1))
adj_r2